# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook and serves as an extension of the API documentation to show what the results will look like depending on what inputs are provided.

In [1]:
from pprint import pprint

import pandas as pd

from wombat.core import Simulation, Metrics

%load_ext autoreload
%autoreload 2

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Table of Contents

Below is a list of top-level sections to demonstrate how to use WOMBAT's `Metrics` class methods and an explanation of each individual metric.
 - [Setup](#setup): Running a simulation to gather the results
 - [Availability](#availability): Time-based and energy-based availability
 - [Capacity Factor](#capacity-factor): Gross and net capacity factor
 - [Task Completion Rate](#task-completion-rate): Task completion metrics
 - [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
 - [Service Equipment Utilization Rate](#service-equipment-utilization-rate): Utilization of servicing equipment
 - [Vessel-Crew Hours at Sea](#vessel-crew-hours-at-sea): Number of crew or vessel hours spent at sea
 - [Number of Tows](#number-of-tows): Number of tows breakdowns
 - [Labor Costs](#labor-costs): Breakdown of labor costs
 - [Equipment and Labor Costs](#equipment-and-labor-costs): Combined servicing equipment and labor cost breakdown
 - [Component Costs](#component-costs): Materials costs
 - [Fixed Cost Impacts](#fixed-cost-impacts): Total fixed costs
 - [OpEx](#opex): Project OpEx
 - [Process Times](#process-times): Timing of various stages of repair and maintenance
 - [Power Production](#power-production): Potential and actually produced power
 - [Net Present Value](#net-present-value): Project NPV calculator
 - [PySAM-Powered Results](#pysam-powered-results): PySAM results, if configuration is provided

<a id="setup"></a>
## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

<a id="availability"></a>
## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 96.6%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.96,0.96,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.97,0.98,0.98,0.98,0.98,0.92,0.98,0.98,0.98,0.98,0.98,0.97,0.98,0.98,0.95,0.98,0.98,0.89,0.95,0.98,0.92,0.98,0.97,0.98,0.96,0.98,0.93,0.97,0.94,0.96,0.93,0.98,0.96,0.98,0.97,0.96,0.98,0.95,0.98,0.95,0.98,0.95,0.91,0.98,0.92,0.95,0.98,0.98,0.98,0.97,0.96,0.98,0.98,0.90,0.96,0.97,0.97,0.98,0.98,0.96,0.98,0.98,0.93,0.98,0.89,0.98,0.97,0.98,0.98,0.98


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.98
2004,0.96
2005,0.95
2006,0.98
2007,0.97
2008,0.95
2009,0.96
2010,0.97
2011,0.97


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.96
2,0.97
3,0.97
4,0.97
5,0.97
6,0.97
7,0.97
8,0.97
9,0.96


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.99
     2          0.99
     3          0.99
     4          0.99
     5          0.99
     6          0.99
     7          0.99
     8          0.98
     9          0.96
     10         0.94
     11         0.94
     12         0.95
2004 1          0.94
     2          0.94
     3          0.95
     4          0.97
     5          0.98
     6          0.99
     7          0.99
     8          0.98
     9          0.96
     10         0.95
     11         0.95
     12         0.92

<a id="capacity-factor"></a>
## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.45,0.46,0.47,0.47,0.47,0.47,0.46,0.47,0.46,0.47,0.46,0.47,0.47,0.46,0.47,0.44,0.47,0.46,0.47,0.46,0.47,0.46,0.47,0.47,0.45,0.47,0.47,0.42,0.45,0.47,0.44,0.47,0.46,0.47,0.46,0.47,0.44,0.47,0.45,0.46,0.45,0.47,0.46,0.47,0.46,0.46,0.47,0.46,0.47,0.45,0.47,0.45,0.43,0.47,0.44,0.46,0.47,0.47,0.46,0.46,0.46,0.47,0.47,0.43,0.46,0.46,0.46,0.47,0.47,0.46,0.47,0.47,0.44,0.47,0.42,0.47,0.46,0.46,0.46,0.47


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.44
2005,0.46
2006,0.46
2007,0.49
2008,0.50
2009,0.45
2010,0.41
2011,0.49


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.51
3,0.51
4,0.42
5,0.39
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.63
     2          0.37
     3          0.38
     4          0.54
     5          0.35
     6          0.31
     7          0.30
     8          0.30
     9          0.27
     10         0.41
     11         0.53
     12         0.53
2004 1          0.53
     2          0.49
     3          0.50
     4          0.39
     5          0.30
     6          0.39
     7          0.27
     8          0.41
     9          0.57
     10         0.54
     11         0.47
     12         0.46

<a id="task-completion-rate"></a>
## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 99%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.99
2004,0.97
2005,1.02
2006,1.00
2007,1.00
2008,1.00
2009,1.01
2010,1.00
2011,0.97


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.96
2,1.04
3,1.04
4,1.02
5,1.02
6,1.00
7,1.01
8,0.99
9,1.00


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

Completion Rate
year month                 
2003 1                 0.87
     2                 1.11
     3                 0.99
     4                 1.01
     5                 0.98
     6                 1.00
     7                 1.03
     8                 0.97
     9                 0.86
     10                1.15
     11                0.98
     12                0.99
2004 1                 0.68
     2                 0.87
     3                 1.39
     4                 1.09
     5                 1.18
     6                 1.07
     7                 0.99
     8                 0.97
     9                 0.94
     10                0.88
     11                0.81
     12                1.00

<a id="equipment-costs"></a>
## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $372,283.50/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","1,724,583.93","68,443,673.61"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"7,270,287.08"
2004,"7,843,736.11"
2005,"13,016,293.26"
2006,"7,050,459.93"
2007,"7,064,430.56"
2008,"7,883,635.92"
2009,"16,912,710.91"
2010,"2,966,250.00"
2011,"12,352,832.25"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,010.42","543,010.42","543,010.42","215,236.90","13,057,282.99"
2,"495,250.00","495,250.00","495,250.00","57,000.00","9,255,404.51"
3,"542,864.58","542,864.58","542,864.58","184,579.72","6,016,958.33"
4,"524,635.42","524,635.42","524,635.42","85,500.00","5,031,250.00"
5,"542,500.00","542,500.00","542,500.00",0.00,"1,190,666.67"
6,"525,000.00","525,000.00","525,000.00",0.00,"6,175,800.35"
7,"542,500.00","542,500.00","542,500.00","74,834.93","5,312,777.78"
8,"542,500.00","542,500.00","542,500.00","214,434.57","2,998,437.50"
9,"525,000.00","525,000.00","525,000.00","342,000.00",0.00


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   50,750.00               50,750.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
     6                   52,500.00                  0.00               0.00  
     7                   54,250.00                  0.00               0.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00            272,479.79               0.00  
     11                  52,500.00              3,166.67       1,000,296.88  
     12                  54,250.00                  0.00       4,076,562.50  
2004 1                   54,250.00                  0.00               0.00  
     2                   50,750.00                  0.00               0.00  
     3                   54,250.00                  0.00       5,089,423.61  
     4                   52,500.00                  0.00         832,812.50  
     5                   54,250.00                  0.00               0.00  
     6                   52,500.00                  0.00               0.00  
     7                   54,250.00                  0.00               0.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00

<a id="service-equipment-utilization-rate"></a>
## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.92,0.96


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.97,0.97
2004,1.00,1.00,1.00,0.00,0.95
2005,1.00,1.00,1.00,0.94,0.97
2006,1.00,1.00,1.00,0.85,1.00
2007,1.00,1.00,1.00,0.00,0.94
2008,1.00,1.00,1.00,0.95,0.94
2009,1.00,1.00,1.00,0.93,0.98
2010,1.00,1.00,1.00,0.00,0.78
2011,1.00,1.00,1.00,0.94,0.95


<a id="vessel-crew-hours-at-sea"></a>
## Vessel-Crew Hours at Sea

The number of vessel hours or crew hours at sea for offshore wind power plant simulations.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed for each vessel simulated
 - `False`: computed only as a sum of all vessels
 
`vessel_crew_assumption`: A dictionary of vessel names (`ServiceEquipment.settings.name`, but also found at `Metrics.service_equipment_names`) and the number of crew onboard at any given time. The application of this assumption transforms the results from vessel hours at sea to crew hours at sea.

In [24]:
# Project total, not broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="project", by_equipment=False)

,Total Crew Hours at Sea
0,"117,530.29"


In [25]:
# Annual project totals, broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="annual", by_equipment=True)

,year,Total Crew Hours at Sea,Crew Transfer Vessel 3,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Field Support Vessel,Heavy Lift Vessel
0,2003,"9,137.55","2,200.25","2,106.25","2,258.75",669.00,"1,903.30"
1,2004,"13,157.81","3,661.25","3,905.75","3,739.25",0.00,"1,851.56"
2,2005,"14,531.33","3,428.50","3,344.75","3,232.75",607.50,"3,917.83"
3,2006,"11,496.00","2,925.00","2,750.25","2,783.00","1,104.00","1,933.75"
4,2007,"11,851.50","3,360.75","3,406.00","3,351.00",0.00,"1,733.75"
5,2008,"12,146.50","2,994.25","2,996.50","2,958.25","1,209.00","1,988.50"
6,2009,"14,069.30","2,677.75","2,868.00","2,573.50",611.00,"5,339.05"
7,2010,"8,574.25","2,811.75","2,910.25","2,852.25",0.00,0.00
8,2011,"12,376.55","2,710.00","2,677.00","2,723.25",695.50,"3,570.80"
9,2012,"10,189.50","3,027.50","2,790.50","2,733.25",0.00,"1,638.25"


<a id="number-of-tows"></a>
## Number of Tows

The number of tows metric will only produce results if any towing actually occurred, otherwise a dataframe with a value of 0 is returned, as is the case in this demonstration.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_tug` options:
 - `True`: computed for each tugboat (towing vessel)
 - `False`: computed only as a sum of all tugboats
 
`by_direction` options:
 - `True`: computed for each direction (to port or to site)
 - `False`: computed as a sum for both directions

In [26]:
# Project Total
# NOTE: This example has no towing, so it will return 0
metrics.number_of_tows(frequency="project")

,total_tows
0,0


<a id="labor-costs"></a>
## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [27]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [28]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [29]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [30]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [31]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

<a id="equipment-and-labor-costs"></a>
## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [32]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,537,117.19"
Repair,"17,583,634.55"
Crew Transfer,"1,038,755.22"
Site Travel,0.00
Mobilization,"6,500,000.00"
Weather Delay,"25,347,747.40"
No Requests,"19,033,781.51"
Not in Shift,"17,307,002.95"


In [33]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,537,117.19","2,537,117.19"
Repair,0.00,0.00,0.00,"17,583,634.55","17,583,634.55"
Crew Transfer,0.00,0.00,0.00,"1,038,755.22","1,038,755.22"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"6,500,000.00","6,500,000.00"
Weather Delay,0.00,0.00,0.00,"25,347,747.40","25,347,747.40"
No Requests,0.00,0.00,0.00,"19,033,781.51","19,033,781.51"
Not in Shift,0.00,0.00,0.00,"17,307,002.95","17,307,002.95"


In [34]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      87,500.00
     Repair        1,310,838.54
     Crew Transfer    87,687.50
     Site Travel           0.00
     Mobilization    500,000.00
     Weather Delay 2,950,299.48
     No Requests   1,234,044.90
     Not in Shift  1,099,916.67
2004 Maintenance     396,648.44
     Repair        1,502,767.36

In [35]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                    
1     Maintenance     232,986.98
      Repair        2,664,634.55
      Crew Transfer   138,346.35
      Site Travel           0.00
      Mobilization  2,000,000.00
      Weather Delay 5,271,289.06
      No Requests   1,593,479.09
      Not in Shift  3,000,815.10
2     Maintenance     214,593.75
      Repair        2,221,237.85

In [36]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          21,947.92
           Crew Transfer    2,552.08
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   18,229.17
           No Requests    113,822.92
           Not in Shift     7,729.17
     2     Maintenance          0.00
           Repair          28,018.23
           Crew Transfer    3,098.96
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   15,239.58
           No Requests     87,281.25
           Not in Shift    13,361.98
     3     Maintenance          0.00
           Repair          25,192.71
           Crew Transfer    2,880.21
           Site Travel          0.00

<a id="component-costs"></a>
## Component Costs

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [37]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
turbine,"97,660,600.48"


In [38]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
turbine   delay                  0.00              0.00   42,519,093.53   
          maintenance            0.00              0.00    2,537,117.19   
          repair                 0.00              0.00   17,583,634.55   

                         total_cost  
component action                     
turbine   delay       42,519,093.53  
          maintenance  2,537,117.19  
          repair      17,583,634.55

In [39]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,turbine,"2,403,500.00",0.00,"5,530,335.94","7,933,835.94"
2004,turbine,"3,532,000.00",0.00,"7,042,663.20","10,574,663.20"
2005,turbine,"5,535,000.00",0.00,"11,385,570.31","16,920,570.31"
2006,turbine,"3,086,000.00",0.00,"5,750,338.54","8,836,338.54"
2007,turbine,"2,959,000.00",0.00,"4,043,666.67","7,002,666.67"
2008,turbine,"3,607,500.00",0.00,"6,241,420.14","9,848,920.14"
2009,turbine,"4,702,000.00",0.00,"11,548,885.42","16,250,885.42"
2010,turbine,"1,741,500.00",0.00,"1,135,768.23","2,877,268.23"
2011,turbine,"3,808,000.00",0.00,"7,725,879.12","11,533,879.12"


In [40]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,turbine,"3,313,500.00",0.00,"11,296,915.80","14,610,415.80"
2,turbine,"4,108,000.00",0.00,"9,611,565.97","13,719,565.97"
3,turbine,"3,172,500.00",0.00,"6,295,171.88","9,467,671.88"
4,turbine,"2,853,500.00",0.00,"3,818,588.54","6,672,088.54"
5,turbine,"1,543,500.00",0.00,"1,678,979.17","3,222,479.17"
6,turbine,"3,445,000.00",0.00,"6,426,914.93","9,871,914.93"
7,turbine,"3,810,000.00",0.00,"5,897,113.72","9,707,113.72"
8,turbine,"1,838,500.00",0.00,"1,524,448.79","3,362,948.79"
9,turbine,"1,581,500.00",0.00,"1,077,018.23","2,658,518.23"


In [41]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     turbine         38,500.00              0.00       48,052.08   
     2     turbine         78,500.00              0.00       58,843.75   
     3     turbine         58,000.00              0.00       40,395.83   
     4     turbine         57,000.00              0.00       36,731.77   
     5     turbine         65,000.00              0.00       51,223.96   
     6     turbine         41,500.00              0.00       36,677.08   
     7     turbine         60,000.00              0.00       39,921.88   
     8     turbine         43,500.00              0.00       48,197.92   
     9     turbine        154,500.00              0.00       90,125.00   
     10    turbine        292,000.00              0.00      233,375.00   
     11    turbine        275,500.00              0.00      611,963.54   
     12    turbine      1,239,500.00              0.00    4,234,828.13   
2004 1     turbine        182,500.00              0.00      161,000.00   
     2     turbine        140,500.00              0.00      152,250.00   
     3     turbine      1,274,000.00              0.00    4,752,173.61   
     4     turbine        556,000.00              0.00      988,562.50   
     5     turbine        236,000.00              0.00      152,031.25   
     6     turbine        112,500.00              0.00       60,338.54   
     7     turbine        131,000.00              0.00       80,773.44   
     8     turbine         94,000.00              0.00       76,908.85   
     9     turbine        218,500.00              0.00      144,411.46   
     10    turbine        225,500.00              0.00      160,088.54   
     11    turbine        172,000.00              0.00      154,000.00   
     12    turbine        189,500.00              0.00      160,125.00   
2005 1     turbine        186,500.00              0.00    1,406,546.88   
     2     turbine      1,149,000.00              0.00    4,022,125.00   
     3     turbine        705,500.00              0.00      266,979.17   
     4     turbine        261,500.00              0.00      154,000.00   

                       total_cost  
year month component               
2003 1     turbine      86,552.08  
     2     turbine     137,343.75  
     3     turbine      98,395.83  
     4     turbine      93,731.77  
     5     turbine     116,223.96  
     6     turbine      78,177.08  
     7     turbine      99,921.88  
     8     turbine      91,697.92  
     9     turbine     244,625.00  
     10    turbine     525,375.00  
     11    turbine     887,463.54  
     12    turbine   5,474,328.13  
2004 1     turbine     343,500.00  
     2     turbine     292,750.00  
     3     turbine   6,026,173.61  
     4     turbine   1,544,562.50  
     5     turbine     388,031.25  
     6     turbine     172,838.54  
     7     turbine     211,773.44  
     8     turbine     170,908.85  
     9     turbine     362,911.46  
     10    turbine     385,588.54  
     11    turbine     326,000.00  
     12    turbine     349,625.00  
2005 1     turbine   1,593,046.88  
     2     turbine   5,171,125.00  
     3     turbine     972,479.17  
     4     turbine     415,500.00

<a id="fixed-cost-impacts"></a>
## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [42]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [43]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,013,230.75"


In [44]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [45]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [46]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


In [47]:
# Project annualized totals at the low level by month
metrics.project_fixed_costs(frequency="annual", resolution="low")

,operations
year,
2003,"1,600,008.00"
2004,"1,604,391.58"
2005,"1,600,008.00"
2006,"1,600,008.00"
2007,"1,600,008.00"
2008,"1,604,391.58"
2009,"1,600,008.00"
2010,"1,600,008.00"
2011,"1,600,008.00"


In [48]:
# Project annualized totals at the low level by year and month
metrics.project_fixed_costs(frequency="month-year", resolution="low").head(28)

operations
year month            
2003 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51
     5      135,891.09
     6      131,507.51
     7      135,891.09
     8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09
2004 1      135,891.09
     2      127,123.92
     3      135,891.09
     4      131,507.51
     5      135,891.09
     6      131,507.51
     7      135,891.09
     8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09
2005 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51

<a id="opex"></a>
## OpEx

Here, we will go through the various input definitions to get the project OpEx

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [49]:
metrics.opex("project")

,OpEx
0,"139,343,269.55"


In [50]:
metrics.opex("annual")

,OpEx
year,
2003,"11,273,795.08"
2004,"12,980,127.70"
2005,"20,151,301.26"
2006,"11,736,467.93"
2007,"11,623,438.56"
2008,"13,095,527.50"
2009,"23,214,718.91"
2010,"6,307,758.00"
2011,"17,760,840.25"


In [51]:
metrics.opex("monthly")

,OpEx
month,
1,"19,573,962.05"
2,"16,146,708.66"
3,"12,361,542.71"
4,"10,859,231.32"
5,"5,720,577.57"
6,"12,510,875.42"
7,"12,184,023.61"
8,"8,037,782.98"
9,"4,813,575.07"


In [52]:
metrics.opex("month-year").head(28)

OpEx
year month             
2003 1       338,672.34
     2       348,240.34
     3       356,641.09
     4       346,007.51
     5       363,641.09
     6       330,507.51
     7       358,641.09
     8       342,141.09
     9       443,507.51
     10      863,120.88
     11    1,567,971.05
     12    5,614,703.59
2004 1       481,141.09
     2       419,873.92
     3     6,662,064.70
     4     1,677,820.01
     5       534,641.09
     6       401,507.51
     7       429,641.09
     8       392,641.09
     9       507,507.51
     10      524,141.09
     11      461,007.51
     12      488,141.09
2005 1     2,230,687.97
     2     5,299,990.34
     3     1,196,533.31
     4       636,007.51

In [53]:
port_fees = pd.DataFrame([], columns=["port_fees"], index=metrics.labor_costs(frequency="month-year", by_type=False).index)
port_fees.fillna(0)

port_fees
year month           
2003 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2004 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2005 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2006 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2007 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2008 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2009 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2010 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2011 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0
2012 1              0
     2              0
     3              0
     4              0
     5              0
     6              0
     7              0
     8              0
     9              0
     10             0
     11             0
     12             0

<a id="process-times"></a>
## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [54]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"233,739.53","84,630.25","76,032.25",584.00
major repair,"83,760.11","1,820.62","1,359.86",26.00
major replacement,"127,738.53","8,154.23","127,738.53",49.00
manual reset,"242,438.82","36,983.75","12,583.00","5,625.00"
medium repair,"11,780.69","9,207.75","7,435.00",184.00
minor repair,"81,165.58","33,667.00","15,619.25","2,344.00"


<a id="power-production"></a>
## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [55]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,677,563,673.50"


In [56]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,677,563,673.50","119,170,633.00","120,383,853.50","123,218,927.00","122,454,239.50","122,720,190.00","122,411,127.00","121,905,788.00","122,410,576.50","122,053,539.50","123,450,809.50","120,782,791.00","122,553,101.00","122,466,214.50","122,222,711.00","123,139,749.00","115,619,104.50","122,800,615.00","122,201,569.50","123,026,404.00","121,738,534.50","123,821,892.50","120,975,584.50","123,655,818.00","122,784,861.00","119,088,997.50","122,522,234.50","123,343,586.50","109,339,337.00","119,548,147.00","123,223,203.00","116,474,023.50","122,750,762.50","121,583,747.50","123,120,375.50","119,724,880.00","123,062,023.50","114,936,956.00","122,466,708.50","117,556,205.50","120,472,143.00","117,062,781.00","123,885,549.00","120,981,515.50","122,449,881.50","121,840,277.00","121,048,315.50","122,868,924.00","119,834,418.50","122,716,633.00","119,619,112.00","123,017,798.00","117,608,008.00","112,898,946.50","123,478,187.50","115,527,263.00","120,108,853.50","122,921,577.00","122,462,797.00","122,077,863.00","120,638,636.00","121,054,163.50","123,408,264.50","122,676,234.50","112,557,967.00","120,333,402.50","121,972,868.50","121,250,141.50","122,988,645.50","123,162,492.50","120,232,030.50","122,327,617.50","122,645,223.50","116,355,493.50","123,335,653.00","111,692,259.00","122,497,931.00","121,759,355.00","121,552,649.50","122,063,706.00","123,468,274.00"


In [57]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"861,489,564.00"
2004,"936,581,067.50"
2005,"968,011,222.50"
2006,"967,204,447.50"
2007,"1,036,781,126.50"
2008,"1,051,797,460.50"
2009,"955,621,037.00"
2010,"868,246,390.00"
2011,"1,027,112,551.50"


In [58]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,072,234,974.00"
2,"825,285,768.00"
3,"908,453,468.00"
4,"717,213,906.00"
5,"693,975,886.00"
6,"560,431,971.00"
7,"589,717,062.50"
8,"648,914,134.00"
9,"794,565,175.00"


In [59]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     112,285,503.50
     2      59,212,107.00
     3      67,749,454.00
     4      93,017,219.00
     5      62,411,418.00
     6      53,988,849.50
     7      53,910,333.00
     8      52,675,405.50
     9      46,351,633.00
     10     73,614,227.50
     11     91,061,580.50
     12     95,211,833.50
2004 1      94,290,426.00
     2      82,617,563.50
     3      89,904,712.50
     4      67,875,729.00
     5      52,769,528.00
     6      67,299,962.50
     7      49,086,930.00
     8      73,495,693.50
     9      98,503,866.50
     10     97,092,665.50
     11     81,626,411.50
     12     82,017,579.00

<a id="net-present-value"></a>
## Net Present Value

Here, we will go through the various input definitions to get the project NPV

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`discount_rate`: The rate of return that could be earned on alternative investments, by default 0.025. 

`offtake_price`: Price of energy, per MWh, by default 80.

In [60]:
metrics.npv("project")

,NPV
0,"199,655,275.50"


In [61]:
metrics.opex("annual")

,OpEx
year,
2003,"11,273,795.08"
2004,"12,980,127.70"
2005,"20,151,301.26"
2006,"11,736,467.93"
2007,"11,623,438.56"
2008,"13,095,527.50"
2009,"23,214,718.91"
2010,"6,307,758.00"
2011,"17,760,840.25"


In [62]:
metrics.opex("monthly")

,OpEx
month,
1,"19,573,962.05"
2,"16,146,708.66"
3,"12,361,542.71"
4,"10,859,231.32"
5,"5,720,577.57"
6,"12,510,875.42"
7,"12,184,023.61"
8,"8,037,782.98"
9,"4,813,575.07"


In [63]:
metrics.opex("month-year").head(28)

OpEx
year month             
2003 1       338,672.34
     2       348,240.34
     3       356,641.09
     4       346,007.51
     5       363,641.09
     6       330,507.51
     7       358,641.09
     8       342,141.09
     9       443,507.51
     10      863,120.88
     11    1,567,971.05
     12    5,614,703.59
2004 1       481,141.09
     2       419,873.92
     3     6,662,064.70
     4     1,677,820.01
     5       534,641.09
     6       401,507.51
     7       429,641.09
     8       392,641.09
     9       507,507.51
     10      524,141.09
     11      461,007.51
     12      488,141.09
2005 1     2,230,687.97
     2     5,299,990.34
     3     1,196,533.31
     4       636,007.51

<a id="pysam-powered-results"></a>
## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [64]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [65]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [66]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [67]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [68]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
